In [3]:
import numpy as np
import math
import re
import itertools
import pandas as pd
import time
import os
import pickle

from Functions import *

SyntaxError: invalid syntax (Functions.py, line 610)

In [56]:
# To merge .pk files

ls_names_met=['mean_experiments', 'max_compounds_per_well', 'n_wells', 'percentage_check', 'mean_extra_exp']
full_dict={}
for j in range(1,5):
    base_dir='diff_'+str(j)
    diff_dict={}
    filenames = next(os.walk(base_dir), (None, None, []))[2]
    for file in filenames:
        full_dir=os.path.join(base_dir,file)
        with open(full_dir, 'rb') as handle:
            f1=pickle.load(handle)
        for i in list(f1):
            if i=='kwargs':
                del f1['kwargs']
                continue
            else:
                WA=assign_wells_mat(i)
                f1[i][1].update({'matrix':WA})
                mean_exp, extra_exp,  _, perc_check= mean_metrics(WA, differentiate=j)
                n_wells=WA.shape[1]
                M_exp=np.round(mean_exp, 2)
                max_comp=np.max(np.sum(WA, axis=0))
                ls_met=[M_exp, max_comp, n_wells, perc_check,  extra_exp]
                f2=f1[i][0].drop(labels='matrix')
                idxs=list(f2.index)
                idxs+=['matrix']
                r_idxs=idxs[-1:]+idxs[:-1]
                dict_met={nm:list(f2[nm])+[val] for nm,val in zip(ls_names_met, ls_met)}

                f1[i][0]=pd.DataFrame(data=dict_met, index=idxs).reindex(r_idxs)

                
                f1[i][0]=f1[i][0].round({'mean_extra_exp':2})
            
        diff_dict.update(f1)
    full_str='Differentiate '+str(j)
    full_dict.update({full_str:diff_dict})

    
    


In [58]:
#output as pickle file
full_dir='Final_precomputed_file.pk'

#with open(full_dir, 'wb') as handle:
    #pickle.dump(full_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [69]:
# To merge files generated with inline_print

all_methods = ['matrix', 'multidim', 'random', 'STD', 'CT','Binary']

results = {}

for method_dir in all_methods:
    filenames = next(os.walk(method_dir), (None, None, []))[2]
    metrics_list = []
    for file in filenames:
        if file.endswith(".txt"):
            try:
                metrics = extract_metrics(file)
                metrics_list.append(metrics)
            except ValueError as e:
                print(e)
    
    if metrics_list:
        df = pd.DataFrame(metrics_list)
        df = df.sort_values(by="NS")
        for diff_value, group_df in df.groupby("diff"):
            group_df = group_df.reset_index(drop=True)
            if diff_value not in results:
                results['Differentiate '+str(diff_value)] = {}
            results['Differentiate '+str(diff_value)][method_dir] = group_df.drop(columns=["diff"])
            

In [ ]:
#output as pickle file
full_dir='Final_inline_precomputed_file.pk'

#with open(full_dir, 'wb') as handle:
    #pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)